<a href="https://colab.research.google.com/github/JacobDowns/glamacles_test/blob/main/train_pinn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install nvidia-modulus nvidia-modulus-sym
! git clone https://github.com/JacobDowns/glamacles_test

  Using cached Cython-0.29.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.37
    Uninstalling Cython-0.29.37:
      Successfully uninstalled Cython-0.29.37
Cloning into 'glamacles_test'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 40 (delta 16), reused 16 (delta 4), pack-reused 0
Receiving objects: 100% (40/40), 46.23 KiB | 3.08 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [12]:
import os
arch = os.getenv("ARGS", "real")

try:
    import google.colab  # noqa: F401
except ImportError:
    import firedrake as fd
else:
    try:
        import firedrake as fd
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/firedrake-install-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake as fd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
cd glamacles_test

/content/glamacles_test/glamacles_test


In [14]:
from model import SpecFO
import numpy as np
import matplotlib.pyplot as plt
data = np.load('/content/drive/MyDrive/glamacles/data.npy')
model = SpecFO()

/usr/local/lib/python3.10/dist-packages/firedrake/interpolation.py:385: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.


In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create input features
B = data[:,0,:,:]
H = data[:,1,:,:]
S = B+H
S_x = np.gradient(S, axis=2)
S_y = np.gradient(S, axis=1)
X = np.stack([H, S_x, S_y], axis=1)

# Output features
Y = data[:,2:,:,:]

# Normalization
X /= X.std(axis=(0,2,3))[np.newaxis,:,np.newaxis,np.newaxis]
Y /= Y.std(axis=(0,2,3))[np.newaxis,:,np.newaxis,np.newaxis]

# Create datsets
N_train = 2000
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

train_dataset = TensorDataset(X[0:N_train], Y[0:N_train])
validation_dataset = TensorDataset(X[N_train:], Y[N_train:])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

In [ ]:
from modulus.models.fno import FNO

# Fourier neural operator model
fno = FNO(
    in_channels=3,
    out_channels=4,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=5,
    num_fno_modes=16,
    padding=9,
).cuda()

optimizer = torch.optim.Adam(fno.parameters(), lr=1e-4)

# Number of epochs
epochs = 100
from torch.utils.tensorboard import SummaryWriter
log_dir = '/content/drive/MyDrive/glamacles/logs/'
writer = SummaryWriter(log_dir=log_dir)

for epoch in range(epochs):
    avg_loss = 0.
    num = 0

    fno.train()
    for i, (x,y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        y_fno = fno(x)
        loss = torch.mean(torch.sqrt((y - y_fno)**2 + 1e-10))
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()
        num += 1

    print('avg_loss', avg_loss / num)
    writer.add_scalar('Loss/train', avg_loss / num, epoch)

    # Validation
    if epoch % 20 == 0:
        with torch.no_grad():

            torch.save(fno.state_dict(), f'/content/drive/MyDrive/glamacles/checkpoint_{epoch}.pt')

            validation_loss = 0.
            for i, (x,y) in enumerate(validation_loader):
                x = x.cuda()
                y = y.cuda()
                y_fno = fno(x)
                loss = torch.mean(torch.sqrt((y - y_fno)**2 + 1e-10))
                validation_loss += loss.item()
                num += 1
            print('validation_loss', validation_loss / num)
            writer.add_scalar('Loss/validation', validation_loss / num, epoch)

avg_loss 0.11760538557803375
validation_loss 0.00934889696855098
avg_loss 0.04837181305838749
avg_loss 0.04375357240985613
avg_loss 0.04027165420312667
avg_loss 0.0378934186209226
avg_loss 0.03538053126144223


In [ ]:
ls